In [1]:
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
ES_API_KEY_ENCODED=json.loads(os.getenv("ES_KEY"))["encoded"]

In [3]:
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# 1. 配置连接
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_base=OPENAI_BASE_URL,
    openai_api_key=OPENAI_API_KEY,
)

In [5]:
llm = ChatOpenAI(
    model = "gpt-4o",
    openai_api_base=OPENAI_BASE_URL,
    openai_api_key=OPENAI_API_KEY,
    temperature= 0.5,
    streaming = True,
)

In [7]:
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore

# 创建 Elasticsearch 客户端
es_client = Elasticsearch(
    hosts=["http://localhost:9200"],
    headers={"Authorization": f"ApiKey {ES_API_KEY_ENCODED}"}
)

# 使用客户端初始化 ElasticsearchStore
db = ElasticsearchStore(
    es_connection=es_client,
    index_name="index1",
    embedding=embeddings
)

In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
loader = TextLoader("./《唐人的餐桌》节选.txt")
documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
db.add_documents(docs)

['e1da02c7-9eac-4769-b41e-dfb4baa8d165',
 '7caba008-c862-47f0-b38e-ea63e0ee386a',
 '6c913855-54f2-4e83-89b6-99e3679c2da6',
 'a88d75b7-225d-46ec-a538-9c57231e4df9',
 'dfd2ddf2-ab77-40b9-afbb-9e24c16f0b45',
 '3f9bf0e4-fcbf-4445-9354-bcdec3cf273a',
 '2d071155-5985-4e8e-aa6c-552ce920f450',
 '35eb9577-723f-47a9-b754-d0ea1fe3312f',
 'ec4c42ff-b5db-4301-9696-35f787614e0d',
 'aa1f5ec6-0ee3-4474-b5e8-c19443b679ee',
 'c5a6b933-f22b-4494-a12c-ed1f8479a501',
 '140ed52d-2909-4db9-843a-ad195ba9c27f',
 '116b8620-b5f7-4ac3-afa6-41eda550f28f',
 '9e2fbb48-e232-464f-b99f-c12fd8284c27',
 'd4b3b7a4-4546-41af-a500-d6771cdf7abe',
 '86ba9905-85a6-4a22-8815-9fcc869620ee',
 'bc6cc3b4-4d03-482e-bd18-82bf80463b3b',
 '9ac8b168-c7cd-4a0d-812e-d8afd2f46e23',
 '63a836cc-2876-429d-aa0d-bab59af2865e',
 'd664f0a8-db9f-41c2-8ac6-85cc4b4e6911',
 '50726cb4-e81d-42f4-9ffb-52c5fa8067a0',
 'd9805405-7c40-4e10-bdc7-b74589e18817',
 '9db1ac4b-3f13-49da-8129-2b2b6e8d2661',
 'eb308ae2-3a9f-4ee8-bdef-58dd4b6ff751',
 'a8db92fa-7756-

In [14]:
query = "清朝人的餐桌上有什么？"
retrieve_context = db.similarity_search(query,k=7)

In [15]:
str(retrieve_context)

"[Document(metadata={'source': './《唐人的餐桌》节选.txt'}, page_content='话，该做的事情一点都不少做。\\n\\u3000\\u3000于是这种菜又被称之为“文贞”菜，因为魏征死后的谥号就是文贞。\\n\\u3000\\u3000大唐官员们相互往来送礼的时候，有一段时间曾经非常盛行赠送这个文贞菜，直到太宗皇帝因为《起居注》的事情差点被气死，用鞭子狠狠抽了魏征墓碑之后，互赠文贞菜这个风潮才慢慢的平缓下来。\\n\\u3000\\u3000云初看了方正拿来的醋芹，吃了一口之后，就立刻断定唐人很可怜，就没有吃过什么好东西，这个醋芹可不是往芹菜里倒醋这样的菜式。\\n\\u3000\\u3000而是先把芹菜烫熟，再往烫芹菜的水里加一点面粉用力的搅拌，最后把芹菜重新倒进面汤里，找一个没有油的坛子装起来，发酵几天之后，这东西就会变得酸香扑鼻。\\n\\u3000\\u3000发酸的面汤很解暑，里面的芹菜经过发酵之后也带着浓郁的酸香味……云初以前的世界里的人们一般把这个东西称之为——浆水！\\n\\u3000\\u3000酒水的口味已经定性，云初没办法更改，至于醋芹，云初有更加高级的吃法。\\n\\u3000\\u3000从桌子上的牛肉块上撕下来几条牛肉，撕成牛肉丝，拍上两颗蒜，放好青葱，最后用西域特有的胡麻油烧热之后泼一下，这样的一盘子醋芹应该有资格上皇帝的餐桌，毕竟，太宗皇帝行述中�'), Document(metadata={'source': './《唐人的餐桌》节选.txt'}, page_content='��级的吃法。\\n\\u3000\\u3000从桌子上的牛肉块上撕下来几条牛肉，撕成牛肉丝，拍上两颗蒜，放好青葱，最后用西域特有的胡麻油烧热之后泼一下，这样的一盘子醋芹应该有资格上皇帝的餐桌，毕竟，太宗皇帝行述中曾经明确记载，他以后的皇帝每年只有三次吃牛肉的机会。\\n\\n\\n第二二章 同流合污\\n\\u3000\\u3000当云初成为唐人之后，再上唐人户籍就是一件理所当然的事情，再加上他现在也在为官府效力，自然在加入户籍这件事上要做到完美才好。\\n\\u3000\\u3000几个人吃完了美味又有名的醋芹，啃了皇帝现在都没办法经常吃的牛肉，又一人用了两碗羊肉面片溜溜缝。\\n\\u3000\\u300

In [16]:
print(retrieve_context[0].page_content)

话，该做的事情一点都不少做。
　　于是这种菜又被称之为“文贞”菜，因为魏征死后的谥号就是文贞。
　　大唐官员们相互往来送礼的时候，有一段时间曾经非常盛行赠送这个文贞菜，直到太宗皇帝因为《起居注》的事情差点被气死，用鞭子狠狠抽了魏征墓碑之后，互赠文贞菜这个风潮才慢慢的平缓下来。
　　云初看了方正拿来的醋芹，吃了一口之后，就立刻断定唐人很可怜，就没有吃过什么好东西，这个醋芹可不是往芹菜里倒醋这样的菜式。
　　而是先把芹菜烫熟，再往烫芹菜的水里加一点面粉用力的搅拌，最后把芹菜重新倒进面汤里，找一个没有油的坛子装起来，发酵几天之后，这东西就会变得酸香扑鼻。
　　发酸的面汤很解暑，里面的芹菜经过发酵之后也带着浓郁的酸香味……云初以前的世界里的人们一般把这个东西称之为——浆水！
　　酒水的口味已经定性，云初没办法更改，至于醋芹，云初有更加高级的吃法。
　　从桌子上的牛肉块上撕下来几条牛肉，撕成牛肉丝，拍上两颗蒜，放好青葱，最后用西域特有的胡麻油烧热之后泼一下，这样的一盘子醋芹应该有资格上皇帝的餐桌，毕竟，太宗皇帝行述中�


In [17]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 将检索结果合并成上下文字符串
context = "\n".join([doc.page_content for doc in retrieve_context])

# 创建聊天提示模板（适配GPT-4o的聊天格式）
system_template = SystemMessagePromptTemplate.from_template(
    "1. 你是一个智能助手。"
    "2. 根据用户问题检索得到的相关文档内容：\n"
    "{context}\n"
    "3. 决定是否需要根据检索得到的相关文档回答问题。\n"
    "4. 如果检索到的内容不相关，必须告诉用户。\n"
    "5. 每次回答必须说出你对用户问题的看法。\n"
    "6. 每次使用检索得到的相关文档，必须标注来源，给出角标，在回答的末尾给出引用的文档。"
    )
human_template = HumanMessagePromptTemplate.from_template("用户问题：{question}")

chat_prompt = ChatPromptTemplate.from_messages([system_template, human_template])

# 格式化提示信息
formatted_prompt = chat_prompt.format_prompt(
    context=context,
    question=query
).to_messages()

# 调用LLM生成答案
result = llm.invoke(formatted_prompt)

# 输出最终答案
print("问题：", query)
print("\n答案：", result.content)

问题： 清朝人的餐桌上有什么？

答案： 目前提供的文档内容主要涉及唐代人的餐桌文化，并未提及清朝人的餐桌内容。因此，我无法直接从文档中找到相关信息。不过，我可以为您提供一些关于清朝饮食文化的常识。

清朝时期的餐桌文化丰富多样，主要受到满族、汉族以及其他少数民族饮食习惯的影响。以下是一些典型的清朝餐桌特色：

1. **满汉全席**：清朝的满汉全席是中国历史上最著名的宴席之一，分为满席和汉席两部分，融合了满族和汉族的饮食精华，菜品多达数十甚至上百道。

2. **主食**：清朝人的主食以米饭、面食为主，北方地区多食面食如饺子、馒头，南方则以米饭为主。

3. **肉类和野味**：清朝皇室和贵族的餐桌上常见鹿肉、熊掌等珍贵野味，此外羊肉、猪肉、鸡肉等也是常见食材。

4. **火锅**：火锅在清朝非常流行，尤其是在宫廷中，是冬季御寒的佳品。

5. **茶文化**：清朝的茶文化极为发达，饮茶成为日常生活的重要部分，常见的有绿茶、红茶、普洱茶等。

6. **点心**：清朝的点心种类繁多，如京八件、酥糖、蜜饯等，尤其是在宫廷中，点心制作非常精致。

如果您想了解更多关于清朝餐桌的具体内容，可以提供更详细的问题，我会尽力为您解答！
